# Résolution du problème du voyageur de commerce mis sous forme QUBO grâce au recuit quantique

##  Sommaire
1. [Modélisation théorique du problème tsp sous forme QUBO](#modélisation)
2. [Code et résultats](#code)


## Modélisation théorique du problème tsp sous forme QUBO


Voyons comment écrire le problème TSP sous forme quadratique pour ensuite l'écrire sous la forme d'une matrice QUBO.
Pour N villes, on utilise $N^{2}$ variables binaires $x(i,j)$ où i représente le numero de la ville et j son ordre de visite dans l'itinéraire du voyageur. Si la ville i et visitée à l'étape j alors on aura $x(i,j)=1$ sinon on aura $x(i,j)=0$. Il faut aussi modéliser le fait que chaque ville est visitée une seule fois (1) et que chaque étape de la route permet de visiter une seule ville (2). Les équations suivantes sont suffisantes pour modéliser ces conditions :

$
\sum_{i=1}^{N}(1-\sum_{j=1}^{N}x(i,j))^{2}=0\quad(1)
$

$
\sum_{j=1}^{N}(1-\sum_{i=1}^{N}x(i,j))^{2}=0\quad(2)
$

De plus on a la matrice D qui encode les distance entre chaque ville, c'est à dire qu'on a $D_{k,l}=D(l,k)=$ distance entre les villes k et l.

On doit maintenant mettre ce problème sous la forme QUBO. Pour cela on doit obtenir une seule matrice encodant les contraintes et les distances entre les villes.

Pour commencer on va chercher à définir une fonction de coût qui sera la fonction à minimiser. Cette fonction doit intégrer les contraintes garantissant un itinéraire valide et la distance totale du trajet.

On a : 

$
f(x)=M\sum_{j=1}^{N}(1-\sum_{i=1}^{N}x(i,j))^{2}+M\sum_{i=1}^{N}(1-\sum_{j=1}^{N}x(i,j))^{2}+\sum_{k,l}D(k,l)\sum_{j=1}^Nx(k,j)x(l,j+1)
$

où M est un grand nombre garantissant que les contraintes sont respectées avant la minimisation de la distance. 



Maintenant, passons à l'étape la plus délicate : cette fonction de coût doit être convertie sous forme matricielle pour être résolue par une approche QUBO.

On doit alors mettre écrire cette fonction de coût sous forme quadratique (plus une constante), c'est à dire sous la forme $f(x)=x^tQx$ où $Q$ est une matrice de taille $N^{2}\times N^{2}$ et $x$ est un vecteur de taille $N^{2}$, tel que 

$
x=\begin{bmatrix}
x(1,1) \\
x(1,2) \\
\vdots \\
x(1,N)\\
x(2,1) \\
x(2,2) \\
\vdots \\
x(2,N)\\
\vdots \\
x(N,1) \\
x(N,2) \\
\vdots \\
x(N,N)\\
\end{bmatrix}
$

 en rappelant que $x(i,j)$ vaut 1 si la ville i est visitée à l'étape j et 0 sinon.\\


On écrit notre matrice Q comme une somme de 3 matrices : $Q=Q_1 + Q_2 + Q_3$ où :

1. $Q_1$ encode le terme de coût de la distance $\sum_{k,l}D(k,l)\sum_{j=1}^Nx(k,j)x(l,j+1)$
2. $Q_2$  est encode la pénalité liée à la contrainte de ne visiter qu'une seule fois chaque ville, c'est à dire $\sum_{i=1}^{N}(1-\sum_{j=1}^{N}x(i,j))^{2}$ 
3. $Q_3$ encode la pénalité liée à la  contrainte de ne visiter qu'une seule ville par étape c'est à dire $\sum_{j=1}^{N}(1-\sum_{i=1}^{N}x(i,j))^{2}$

### Calcul de $Q_2$

Pour obtenir $Q_2$, on doit mettre $\sum_{i=1}^{N}(1-\sum_{j=1}^{N}x(i,j))^{2}$ sous forme quadratique.

On a $\sum_{i=1}^{N}(1-\sum_{j=1}^{N}x(i,j))^{2}=N - 2 \sum_{i=1}^{N} \sum_{j=1} x(i,j) +  \sum_{i=1}^{N} \left( \sum_{j=1} x(i,j) \right)^2
$

On va décomposer notre matrice $Q_2$ en une somme de deux matrices $Q_2=K_2+R_2$ où $K_2$ encode $- 2 \sum_{i=1}^{N} \sum_{j=1} x(i,j) $ et $R_2$ encode $\sum_{i=1}^{N} \left( \sum_{j=1} x(i,j) \right)^2$.

Tout d'abord $K_2$ est de taille $N^{2}\times N^{2}$ telle que $K_2=\begin{pmatrix}
-2 & 0 & 0 & \cdots & 0 \\
0 & -2 & 0 & \cdots & 0 \\
0 & 0 & -2 & \cdots & 0 \\
\vdots & \vdots & \vdots & \ddots & \vdots \\
0 & 0 & 0 & \cdots & -2
\end{pmatrix}
$

$R_2$ est un peu plus complexe.

$R_2$ est une matrice de taille $N^{2}\times N^{2}$ composée de N blocs diagonaux de taille $N\times N$ remplis de 1 et dont les coefficients sont égaux à 0 en dehors de ces blocs remplis de 1.

Par exemple, pour N=3, cela nous donne :

$R_2=\begin{pmatrix}
1 & 1 & 1 & 0 & 0 & 0 & 0 & 0 & 0 \\
1 & 1 & 1 & 0 & 0 & 0 & 0 & 0 & 0 \\
1 & 1 & 1 & 0 & 0 & 0 & 0 & 0 & 0 \\
0 & 0 & 0 & 1 & 1 & 1 & 0 & 0 & 0 \\
0 & 0 & 0 & 1 & 1 & 1 & 0 & 0 & 0 \\
0 & 0 & 0 & 1 & 1 & 1 & 0 & 0 & 0 \\
0 & 0 & 0 & 0 & 0 & 0 & 1 & 1 & 1 \\
0 & 0 & 0 & 0 & 0 & 0 & 1 & 1 & 1 \\
0 & 0 & 0 & 0 & 0 & 0 & 1 & 1 & 1 \\
\end{pmatrix}$

En effet on a bien :

$x^t R_2 x  = x^t\left( \begin{matrix} 
\sum_j x(1,j) \\
\vdots \\
\sum_j x(2,j) \\
\vdots \\
\sum_j x(N,j) \\
\end{matrix} \right) = \left(  \sum_{i=1}^N \left( \sum_{j=1} x(i,j) \right)^2 \right)
$

La somme de ces deux matrices nous donne $Q_2$

### Calcul de $Q_3$

De même que pour $Q_2 $ mais avec i et j interchangés on a :

$\sum_{j=1}^{N}(1-\sum_{i=1}^{N}x(i,j))^{2}=N - 2 \sum_{j=1}^{N} \sum_{i=1} x(i,j) +  \sum_{j=1}^{N} \left( \sum_{i=1} x(i,j) \right)^2
$

On décompose en somme de matrice de la même manière que pour $Q_2$ d'où :

$
Q_3=K_3+R_3
$

On a $K_3=K_2$ et $R_3$ est une matrice de taille $N^{2}\times N^{2}$ composée de $N\times N$ blocs égaux à la matrice identité de taille $N\times N$. Par exemple, pour N=3 cela nous donne :

$R_3=\begin{pmatrix}
1 & 0 & 0 & 1 & 0 & 0 & 1 & 0 & 0 \\
0 & 1 & 0 & 0 & 1 & 0 & 0 & 1 & 0 \\
0 & 0 & 1 & 0 & 0 & 1 & 0 & 0 & 1 \\
1 & 0 & 0 & 1 & 0 & 0 & 1 & 0 & 0 \\
0 & 1 & 0 & 0 & 1 & 0 & 0 & 1 & 0 \\
0 & 0 & 1 & 0 & 0 & 1 & 0 & 0 & 1 \\
1 & 0 & 0 & 1 & 0 & 0 & 1 & 0 & 0 \\
0 & 1 & 0 & 0 & 1 & 0 & 0 & 1 & 0 \\
0 & 0 & 1 & 0 & 0 & 1 & 0 & 0 & 1 \\
\end{pmatrix}$\\

On obtient bien :

$
x^tR_3x=x^t\left( 
\begin{array}{c}
\sum_i x(i,1) \\
\vdots \\
\sum_i x(i,N) \\
\sum_i x(i,1) \\
\vdots \\
\sum_i x(i,N) \\
\sum_i x(i,1) \\
\vdots \\
\sum_i x(i,N)
\end{array}
\right)
=  \sum_{j=1}^N \left( \sum_{i=1}^N x(i,j) \right)^2.
$

### Calcul de $Q_1$

Rappelons que $Q_1$ doit encoder le calcul de la distance parcourue en fonction de l'itinéraire choisie.

Tout d'abord, la distance entre une ville k et une ville l est comptée seulement si dans l'itinéraire on visite la ville l juste après la ville k. Ceci est représenté par la somme $D(k,l)\sum_{j=1}^{N-1}x(k,j)x(l,j+1)$ où D est la matrice des distances entre les villes.

On somme sur toutes les villes pour obtenir le coût total de déplacement c'est à dire :

$\sum_{k,l}\sum_{j=1}^{N}x(k,j)x(l,j+1)$

C'est déja sous forme quadratique donc il n'y a plus qu'à l'écrire sous forme de matrice. 
On obtient :

$
Q_1 = 
\begin{pmatrix}
\mathbf{M}(1,1) & \mathbf{M}(1,2) & \cdots & \mathbf{M}(1,N) \\
\mathbf{M}(2,1) & \mathbf{M}(2,2) & \cdots & \mathbf{M}(2,N) \\
\vdots & \vdots & \ddots & \vdots \\
\mathbf{M}(N,1) & \mathbf{M}(N,2) & \cdots & \mathbf{M}(N,N)
\end{pmatrix}
$

où 

$
M(k,l) = 
\begin{pmatrix}
0 & D(k,l) & 0 \cdots & 0  & D(l,k) \\
0 & 0 & D(k,l) & \cdots & 0 \\
0 & 0 & 0 & \cdots & 0 \\
\vdots & \vdots & \vdots & \ddots & \vdots \\
0 & 0 & 0 & \cdots & D(k,l) \\
0 & 0 & 0 & \cdots & 0 \\
\end{pmatrix}
$


$D(l,k)$ dans le coin inférieur droit de chaque bloc représente le coût de retourné à la première ville visitée depuis la dernière ville visitée.









### Résumé 

On doit donc minimiser la forme quadratique $x^tQx$ où 

$Q=Q_1+MQ_2+MQ_3=Q_1+M(K_2+R_2)+M(K_3+R_3)$

Pour incorporer des contraintes dans le problème QUBO et être sûre de les respecter, il faut multiplier les matrices des contraintes par un grand nombre M tel que M soit du même ordre de grandeur que \( \max_{k,l} D(k,l) N^2 \). De cette manière, le problème QUBO ne commence a être minimisé que lorsque les contraintes sont satisfaites, car le coût de ne pas satisfaire une contrainte est plus grand que tout coût associé au trajet. Nous verrons que comme l'implémentation ne marchait pas avec cette méthode, au lieu de multiplier les contraintes par un grand scalaire M nous avons normalisé la matrice des distances.







## Code et résultats

Nous avons d'abord tenté d'implémenter la modélisation présentée ci dessus telle qu'elle mais le code ne marchait pas. Plusieurs modifications par rapport à la modélisation théorique ont alors dû être faites. Les deux principales modifications sont les suivantes :

1.Comme MyQlm cherche à maximiser la quantité $x^{t}Qx$ on doit donc lui donner la matrice la matrice -Q et non la matrice Q de notre optimisation, on remplace alors les plus par des moins et les moins par des plus.

2.Au lieu de multiplier les matrices des contraintes par un scalaire M pour être sûre que les contraintes soient respectées, on normalise notre matrice des distances, ce qui revient à peu près au même.



### Calcul des distances entres les villes 

Tout d'abord nous implémentons les focntions nécessaires au calcul de la matrice des distances entre les villes à partir d'un fichiers .csv. Nous convertissons les distances obtenues pour en avoir une approximation en kilomètres.

In [1]:
globals().clear()  # Supprime toutes les variables globales
locals().clear()
import os
import pandas as pd
import numpy as np
from qat.core.variables import Variable
from qat.opt import QUBO
from qat.qpus import SimulatedAnnealing
from qat.simulated_annealing import integer_to_spins
from math import radians, sin, cos, sqrt, atan2
def euclidean_distance_km(lat1, lon1, lat2, lon2):
    R = 111  # Approximation : 1 degré ≈ 111 km en latitude
    dlat = (lat2 - lat1) * R  # Conversion de la latitude en km
    dlon = (lon2 - lon1) * R * np.cos(np.radians((lat1 + lat2) / 2))  # Correction en longitude
    return np.sqrt(dlat**2 + dlon**2)

# Calcul de la matrice des distances avec conversion en kilomètres
def calculer_matrice_distances(coords):
    n = len(coords)
    matrice_distances_normalisee = np.zeros((n, n))

    for i in range(n):
        for j in range(n):
            if i != j:
                matrice_distances_normalisee[i, j] = euclidean_distance_km(
                    coords[i][0], coords[i][1], coords[j][0], coords[j][1]
                )

    max_distance = np.max(matrice_distances_normalisee)
    matrice_distances_originale = matrice_distances_normalisee
    matrice_distances_normalisee = matrice_distances_normalisee / max_distance  # Normalisation de la matrice des distances

    return matrice_distances_normalisee, matrice_distances_originale
    
    
    
    

### Construction de la matrice QUBO

In [2]:
def probleme_tsp(graphe):
    N = graphe.shape[0] 
    taille_Q = N * N  
    matrice_Q = np.zeros((taille_Q, taille_Q))  
    

    # Implémentation de la contrainte chaque ville doit être visitée exactement une fois
    for i in range(n):
        for j1 in range(n):
            for j2 in range(n):
                if j1 < j2:
                    
                    matrice_Q[i * n + j1, i * n + j2] += -2
                    matrice_Q[i * n + j2, i * n + j1] += -2 
        for j in range(n):
            
            matrice_Q[i * n + j, i * n + j] += -1
            

    # Implémentation de la contrainte un seul déplacement par étape
    for j in range(n):
        for i1 in range(n):
            for i2 in range(n):
                if i1 < i2:
                    
                    matrice_Q[i1 * n + j, i2 * n + j] += -2
                    matrice_Q[i2 * n + j, i1 * n + j] += -2 

        for i in range(n):
            
            matrice_Q[i * n + j, i * n + j] += -1
           
            


    # Implémentation des distances dans le QUBO
    for i in range(n):
        for k in range(n):
            if i != k:
                for j in range(n - 1):  
                    idx1 = i * n + j
                    idx2 = k * n + (j + 1)
                    matrice_Q[idx1, idx2] -= matrice_distances_normalisee[i, k]
                    #matrice_Q[idx1, n] -= matrice_distances_normalisee[i, k]  

    
    np.fill_diagonal(matrice_Q, matrice_Q.diagonal() + taille_probleme / 2)

    # rendre Q symmétrique
    Q = (matrice_Q + matrice_Q.T) / 2

    return QUBO(Q, offset_q=0)

### Implémentation des fonctions pour calculer la distance parcourue et pour afficher le parcours

In [3]:
def afficher_parcours_villes(solution, n):
    
    parcours = np.argmax(solution.reshape((n, n)), axis=0).tolist()
    villes_parcourues = [labels[i] for i in parcours]
    print( "Parcours : ", " -> ".join(villes_parcourues))

def calculer_distance_parcourue(matrice_distances_normalisee, solution):
    
    n = int(np.sqrt(len(solution)))  
    solution_matrix = np.array(solution).reshape((n, n))
    parcours = np.argmax(solution_matrix, axis=0).tolist()  
    
    distance_totale = 0
    for i in range(n - 1):
        distance_totale += matrice_distances_normalisee[parcours[i], parcours[i+1]]
    
    
    distance_totale += matrice_distances_normalisee[parcours[-1], parcours[0]]
    
    return distance_totale


### Implémentation du main

In [4]:



os.chdir('/home/bauc/Documents')

graphe_exemple = np.genfromtxt("villes.csv", delimiter=",")
adjacence = graphe_exemple[1:, 1:]
print(f"adjacence : {adjacence}")

n = adjacence.shape[0]
print(f"n : {n}")
taille_probleme = n*n  # À compléter

temperature_min = 0.001
temperature_max = 1

nombre_etapes = 1000
penalite = 1

data = pd.read_csv("villes.csv")


villes = data[["latitude", "longitude"]].values
labels = data["label"].values
N = len(villes)  



matrice_distances_normalisee,matrice_distances_originale = calculer_matrice_distances(adjacence)


def temperature_lineaire(temp_min, temp_max):
    t = Variable("t", float)
    return temp_min * t + temp_max * (1 - t)

probleme_QUBO = probleme_tsp(matrice_distances_normalisee)
probleme_QUBO_job = probleme_QUBO.sqa_job()


temperature = temperature_lineaire(temp_min=temperature_min, temp_max=temperature_max)
solver = SimulatedAnnealing(temp_t=temperature, n_steps=nombre_etapes)


resultat = solver.submit_job(probleme_QUBO_job)


etat = resultat.raw_data[0].state.int
solution = integer_to_spins(etat, taille_probleme)
    
print("Solution：",solution)


afficher_parcours_villes(solution, n) 


print("Coût:", calculer_distance_parcourue(matrice_distances_originale, solution))

adjacence : [[49.44149568  1.09357554]
 [48.11168041 -1.68186872]
 [43.61335437  3.86926405]
 [48.571265    7.76776038]
 [47.87350373  1.91731602]]
n : 5


/tmp/ipykernel_38055/1322571703.py:34: UserWarning: If SQAQPU is used, gamma_t will be needed.
  probleme_QUBO_job = probleme_QUBO.sqa_job()


Solution： [-1. -1.  1. -1. -1. -1. -1. -1. -1.  1.  1. -1. -1. -1. -1. -1.  1. -1.
 -1. -1. -1. -1. -1.  1. -1.]
Parcours :  montpellier -> strasbourg -> rouen -> orleans -> rennes
Coût: 2233.5642447640444


### Retour sur les résultats obtenus 

Par exemple, sur le fichier test fourni nous obtenons le parcours suivant : strasbourg -> orleans -> rouen -> rennes -> montpellier, dont la distance est de 2159 km.